<a href="https://colab.research.google.com/github/duonghung86/Vehicle-trajectory-tracking/blob/master/Codes/VTP_1_10_Models_for_full_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Developing models for full dataset
---

Overview on the configuration of all simple models

|Model name|Description| # Car|# vars|# targets|# const_vars| # steps | # futures
|---|:--|:-:|:-:|:-:|:-:|:-:|:-:|
Model 1|Baseline model with multi input time steps|1|1|1|0|**4**|1| 
Model 2|Add multi output time steps|1|1|1|0|4|**2**|
Model 3|Add multi input series|1|**2**|1|0|4|2|
Model 4|Add multi `output` series|1|2|**2**|0|4|2|
Model 5|Add multi constant input|1|2|2|**2**|4|2|
Model 6|Add more objects|**4**|2|2|2|4|2|


## Import packages

In [1]:
# For general
import matplotlib.pyplot as plt
import numpy as np
import time
plt.rcParams['figure.figsize'] = (8, 6)

# For data processing
import pandas as pd
from sklearn.model_selection import train_test_split

# For prediction model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

## Load dataset

In [2]:
url_1 = 'https://github.com/duonghung86/Vehicle-trajectory-tracking/raw/master/Data/NGSIM/0750_0805_us101_smoothed_11_.zip'
zip_path = tf.keras.utils.get_file(origin=url_1, fname=url_1.split('/')[-1], extract=True)
csv_path = zip_path.replace('zip','csv')
csv_path

'C:\\Users\\Duong Hung\\.keras\\datasets\\0750_0805_us101_smoothed_11_.csv'

Let's take a glance at the data. Here are the first few rows:

In [3]:
df = pd.read_csv(csv_path)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 18 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   Vehicle_ID    1048575 non-null  int64  
 1   Frame_ID      1048575 non-null  int64  
 2   Total_Frames  1048575 non-null  int64  
 3   Global_Time   1048575 non-null  int64  
 4   Local_X       1048575 non-null  float64
 5   Local_Y       1048575 non-null  float64
 6   Global_X      1048575 non-null  float64
 7   Global_Y      1048575 non-null  float64
 8   v_Length      1048575 non-null  float64
 9   v_Width       1048575 non-null  float64
 10  v_Class       1048575 non-null  int64  
 11  v_Vel         1048575 non-null  float64
 12  v_Acc         1048575 non-null  float64
 13  Lane_ID       1048575 non-null  int64  
 14  Preceeding    1048575 non-null  int64  
 15  Following     1048575 non-null  int64  
 16  Space_Hdwy    1048575 non-null  float64
 17  Time_Hdwy     1048575 non-n

,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_Length,v_Width,v_Class,v_Vel,v_Acc,Lane_ID,Preceeding,Following,Space_Hdwy,Time_Hdwy
0,2,13,437,1118846980200,16.467196,35.380427,6451137.641,1873344.962,14.5,4.9,2,40.000000,0.000000,2,0,0,0.0,0.0
1,2,14,437,1118846980300,16.446594,39.381608,6451140.329,1873342.000,14.5,4.9,2,40.012349,0.123485,2,0,0,0.0,0.0
2,2,15,437,1118846980400,16.425991,43.381541,6451143.018,1873339.038,14.5,4.9,2,39.999855,-0.124939,2,0,0,0.0,0.0
3,2,16,437,1118846980500,16.405392,47.380780,6451145.706,1873336.077,14.5,4.9,2,39.992920,-0.069349,2,0,0,0.0,0.0
4,2,17,437,1118846980600,16.384804,51.379881,6451148.395,1873333.115,14.5,4.9,2,39.991544,-0.013759,2,0,0,0.0,0.0


Next look at the statistics of the dataset:

In [4]:
df.describe().transpose().round(3)

,count,mean,std,min,25%,50%,75%,max
Vehicle_ID,1048575.0,1.533080e+03,790.271,2.000000e+00,9.320000e+02,1.574000e+03,2.210000e+03,2.783000e+03
Frame_ID,1048575.0,4.518249e+03,2412.479,8.000000e+00,2.455000e+03,4.586000e+03,6.598000e+03,8.906000e+03
Total_Frames,1048575.0,5.608770e+02,146.577,1.770000e+02,4.640000e+02,5.180000e+02,6.400000e+02,1.010000e+03
Global_Time,1048575.0,1.118847e+12,241247.914,1.118847e+12,1.118847e+12,1.118847e+12,1.118848e+12,1.118848e+12
Local_X,1048575.0,2.940600e+01,16.666,5.340000e-01,1.728400e+01,2.955700e+01,4.187500e+01,7.347800e+01
Local_Y,1048575.0,1.002056e+03,596.357,1.796600e+01,4.883960e+02,9.640280e+02,1.491548e+03,2.195470e+03
Global_X,1048575.0,6.451838e+06,446.275,6.451107e+06,6.451450e+06,6.451808e+06,6.452205e+06,6.452734e+06
Global_Y,1048575.0,1.872677e+06,397.006,1.871875e+06,1.872352e+06,1.872699e+06,1.873015e+06,1.873365e+06
v_Length,1048575.0,1.463500e+01,4.870,4.000000e+00,1.200000e+01,1.450000e+01,1.650000e+01,7.610000e+01
v_Width,1048575.0,6.132000e+00,1.037,2.000000e+00,5.400000e+00,6.000000e+00,6.900000e+00,8.500000e+00


In [5]:
df.columns

Index(['Vehicle_ID', 'Frame_ID', 'Total_Frames', 'Global_Time', 'Local_X',
       'Local_Y', 'Global_X', 'Global_Y', 'v_Length', 'v_Width', 'v_Class',
       'v_Vel', 'v_Acc', 'Lane_ID', 'Preceeding', 'Following', 'Space_Hdwy',
       'Time_Hdwy'],
      dtype='object')

In [6]:
#  keep only columns that are useful for now
kept_cols = ['Vehicle_ID', 'Frame_ID', 'Total_Frames', 'Local_X','Local_Y','v_Length', 'v_Width', 'v_Class',
       'v_Vel', 'v_Acc', 'Lane_ID']
df = df[kept_cols]
df.head()

,Vehicle_ID,Frame_ID,Total_Frames,Local_X,Local_Y,v_Length,v_Width,v_Class,v_Vel,v_Acc,Lane_ID
0,2,13,437,16.467196,35.380427,14.5,4.9,2,40.000000,0.000000,2
1,2,14,437,16.446594,39.381608,14.5,4.9,2,40.012349,0.123485,2
2,2,15,437,16.425991,43.381541,14.5,4.9,2,39.999855,-0.124939,2
3,2,16,437,16.405392,47.380780,14.5,4.9,2,39.992920,-0.069349,2
4,2,17,437,16.384804,51.379881,14.5,4.9,2,39.991544,-0.013759,2


In [7]:
'the number of vehicles is {}'.format(len(df.Vehicle_ID.unique()))

'the number of vehicles is 1993'

# Model 1

### Constant values

In [8]:
n_steps = 3
n_future = 1
n_features = len(df)
series_feature_names = ['Local_X','v_Vel','Local_Y', 'v_Acc', 'Lane_ID']
target_names = ['Local_X','v_Vel']
n_labels = len(target_names)
vehicle_ids = df.Vehicle_ID.unique()

## Data preparation

### `series2seq`: Function that return sequence input and output for one object

**Arguments**:

- data: Sequence of observations as a Pandas dataframe.
- n_in: Number of lag observations as input (X).
- n_out: Number of observations as output (y).
- **series_features**: names of series features
- labels: name of target variables
- dropnan: Boolean whether or not to drop rows with NaN values.
    
**Returns**:
- X: Feature Pandas DataFrame
- y: Label Pandas dataframe

  

In [9]:
# Test data frame
# data set of the vehicle #2
df_test = df[df.Vehicle_ID==vehicle_ids[0]].copy()
df_test.info()
df_test.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 437 entries, 0 to 436
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Vehicle_ID    437 non-null    int64  
 1   Frame_ID      437 non-null    int64  
 2   Total_Frames  437 non-null    int64  
 3   Local_X       437 non-null    float64
 4   Local_Y       437 non-null    float64
 5   v_Length      437 non-null    float64
 6   v_Width       437 non-null    float64
 7   v_Class       437 non-null    int64  
 8   v_Vel         437 non-null    float64
 9   v_Acc         437 non-null    float64
 10  Lane_ID       437 non-null    int64  
dtypes: float64(6), int64(5)
memory usage: 41.0 KB


,Vehicle_ID,Frame_ID,Total_Frames,Local_X,Local_Y,v_Length,v_Width,v_Class,v_Vel,v_Acc,Lane_ID
0,2,13,437,16.467196,35.380427,14.5,4.9,2,40.000000,0.000000,2
1,2,14,437,16.446594,39.381608,14.5,4.9,2,40.012349,0.123485,2
2,2,15,437,16.425991,43.381541,14.5,4.9,2,39.999855,-0.124939,2
3,2,16,437,16.405392,47.380780,14.5,4.9,2,39.992920,-0.069349,2
4,2,17,437,16.384804,51.379881,14.5,4.9,2,39.991544,-0.013759,2


In [10]:
def series2seq(data, n_in=1, n_out=1,labels=None,series_features=None, show_result=False):
    dat = data.copy()
    
    features = dat.columns
    targets = labels
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)

    for i in range(n_in, 0, -1):
        cols.append(dat[series_features].shift(i))
        names += ['{}(t-{})'.format(j, i) for j in series_features]
    # forecast sequence (t, t+1, ... t+n) for selected labels
    #print(targets)
    for i in range(0, n_out):
        cols.append(dat[targets].shift(-i))
        names += ['{}(t+{})'.format(j, i) for j in targets]
    # put it all together
    agg = pd.concat(cols, axis=1).dropna()
    agg.columns = names
    # concatenate with constant features

    X = agg.iloc[:,:len(series_features)*n_in]
    X = pd.concat([X,dat.drop(columns=series_features)], axis=1).dropna()
    y = agg.iloc[:,len(series_features)*n_in:].copy()
    if show_result:
      X.info()
      print(X.head(), X.shape)
      y.info()
      print(y.head(), y.shape)
    return X, y
  
# Test the function
X, y = series2seq(df_test, n_in=2, n_out=1,labels = target_names,series_features=series_feature_names, show_result=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 435 entries, 2 to 436
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Local_X(t-2)  435 non-null    float64
 1   v_Vel(t-2)    435 non-null    float64
 2   Local_Y(t-2)  435 non-null    float64
 3   v_Acc(t-2)    435 non-null    float64
 4   Lane_ID(t-2)  435 non-null    float64
 5   Local_X(t-1)  435 non-null    float64
 6   v_Vel(t-1)    435 non-null    float64
 7   Local_Y(t-1)  435 non-null    float64
 8   v_Acc(t-1)    435 non-null    float64
 9   Lane_ID(t-1)  435 non-null    float64
 10  Vehicle_ID    435 non-null    int64  
 11  Frame_ID      435 non-null    int64  
 12  Total_Frames  435 non-null    int64  
 13  v_Length      435 non-null    float64
 14  v_Width       435 non-null    float64
 15  v_Class       435 non-null    int64  
dtypes: float64(12), int64(4)
memory usage: 57.8 KB
   Local_X(t-2)  v_Vel(t-2)  Local_Y(t-2)  v_Acc(t-2)  Lane_ID(t-2)  

### `treatment_cars` Function to prepare the data set for each car

In [11]:
# Test data frame
# data set of the first 5 vehicles
df_test = df[df.Vehicle_ID.isin(vehicle_ids[:5])].copy()
df_test.info()
print(df_test.Vehicle_ID.unique())
df_test.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2045 entries, 0 to 2044
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Vehicle_ID    2045 non-null   int64  
 1   Frame_ID      2045 non-null   int64  
 2   Total_Frames  2045 non-null   int64  
 3   Local_X       2045 non-null   float64
 4   Local_Y       2045 non-null   float64
 5   v_Length      2045 non-null   float64
 6   v_Width       2045 non-null   float64
 7   v_Class       2045 non-null   int64  
 8   v_Vel         2045 non-null   float64
 9   v_Acc         2045 non-null   float64
 10  Lane_ID       2045 non-null   int64  
dtypes: float64(6), int64(5)
memory usage: 191.7 KB
[2 4 5 6 8]


,Vehicle_ID,Frame_ID,Total_Frames,Local_X,Local_Y,v_Length,v_Width,v_Class,v_Vel,v_Acc,Lane_ID
0,2,13,437,16.467196,35.380427,14.5,4.9,2,40.000000,0.000000,2
1,2,14,437,16.446594,39.381608,14.5,4.9,2,40.012349,0.123485,2
2,2,15,437,16.425991,43.381541,14.5,4.9,2,39.999855,-0.124939,2
3,2,16,437,16.405392,47.380780,14.5,4.9,2,39.992920,-0.069349,2
4,2,17,437,16.384804,51.379881,14.5,4.9,2,39.991544,-0.013759,2


In [12]:
def treatment_cars(data, n_in=1, n_out=1,labels=None,series_features=None, show_result=False):
  veh_ids = data.Vehicle_ID.unique()
  dat_X, dat_y = pd.DataFrame(),pd.DataFrame()

  for id in veh_ids:
    dat = data[data.Vehicle_ID==id].copy()
    X, y = series2seq(dat.drop(columns=['Frame_ID']), n_in=n_in, n_out=n_out,labels = labels,series_features=series_features)
    dat_X = pd.concat([dat_X,X],ignore_index=True)
    dat_y = pd.concat([dat_y,y],ignore_index=True)
  if show_result:
    dat_X.info()
    print(dat_X.head(), dat_X.shape)
    dat_y.info()
    print(dat_y.head(), dat_y.shape)
  return dat_X ,dat_y
treatment_cars(df_test,n_in=2, n_out=1, labels = target_names,series_features=series_feature_names, show_result=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2035 entries, 0 to 2034
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Local_X(t-2)  2035 non-null   float64
 1   v_Vel(t-2)    2035 non-null   float64
 2   Local_Y(t-2)  2035 non-null   float64
 3   v_Acc(t-2)    2035 non-null   float64
 4   Lane_ID(t-2)  2035 non-null   float64
 5   Local_X(t-1)  2035 non-null   float64
 6   v_Vel(t-1)    2035 non-null   float64
 7   Local_Y(t-1)  2035 non-null   float64
 8   v_Acc(t-1)    2035 non-null   float64
 9   Lane_ID(t-1)  2035 non-null   float64
 10  Vehicle_ID    2035 non-null   int64  
 11  Total_Frames  2035 non-null   int64  
 12  v_Length      2035 non-null   float64
 13  v_Width       2035 non-null   float64
 14  v_Class       2035 non-null   int64  
dtypes: float64(12), int64(3)
memory usage: 238.6 KB
   Local_X(t-2)  v_Vel(t-2)  Local_Y(t-2)  v_Acc(t-2)  Lane_ID(t-2)  \
0     16.467196   40.000000     35.380

(      Local_X(t-2)  v_Vel(t-2)  Local_Y(t-2)  v_Acc(t-2)  Lane_ID(t-2)  \
 0        16.467196   40.000000     35.380427    0.000000           2.0   
 1        16.446594   40.012349     39.381608    0.123485           2.0   
 2        16.425991   39.999855     43.381541   -0.124939           2.0   
 3        16.405392   39.992920     47.380780   -0.069349           2.0   
 4        16.384804   39.991544     51.379881   -0.013759           2.0   
 ...            ...         ...           ...         ...           ...   
 2030     44.212485   61.023160   2092.082904   -2.031700           4.0   
 2031     44.206021   60.569662   2098.139867   -4.534983           4.0   
 2032     44.201809   60.005353   2104.140401   -5.643088           4.0   
 2033     44.197358   59.879807   2110.128380   -1.255460           4.0   
 2034     44.193051   59.887172   2116.117096    0.073649           4.0   
 
       Local_X(t-1)  v_Vel(t-1)  Local_Y(t-1)  v_Acc(t-1)  Lane_ID(t-1)  \
 0        16.446594   4

### Choose the size of the raw data set

In [13]:
np.random.seed(23)
new_size = 100
veh_list = np.random.choice(df.Vehicle_ID.unique(),new_size)
sub_df = df[df.Vehicle_ID.isin(veh_list)].copy()
sub_df.info()
sub_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50149 entries, 6303 to 1040359
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Vehicle_ID    50149 non-null  int64  
 1   Frame_ID      50149 non-null  int64  
 2   Total_Frames  50149 non-null  int64  
 3   Local_X       50149 non-null  float64
 4   Local_Y       50149 non-null  float64
 5   v_Length      50149 non-null  float64
 6   v_Width       50149 non-null  float64
 7   v_Class       50149 non-null  int64  
 8   v_Vel         50149 non-null  float64
 9   v_Acc         50149 non-null  float64
 10  Lane_ID       50149 non-null  int64  
dtypes: float64(6), int64(5)
memory usage: 4.6 MB


,Vehicle_ID,Frame_ID,Total_Frames,Local_X,Local_Y,v_Length,v_Width,v_Class,v_Vel,v_Acc,Lane_ID
6303,25,77,436,36.937434,39.958126,18.5,5.9,2,43.640000,0.000000,4
6304,25,78,436,36.913762,44.531105,18.5,5.9,2,45.730403,20.904029,4
6305,25,79,436,36.890084,49.025028,18.5,5.9,2,44.939855,-7.905483,4
6306,25,80,436,36.866466,53.459789,18.5,5.9,2,44.348240,-5.916150,4
6307,25,81,436,36.842977,57.855282,18.5,5.9,2,43.955558,-3.926819,4


In [14]:
%%time
X, y = treatment_cars(sub_df, 
                   n_in=n_steps, n_out=n_future,
                   labels = target_names,
                   series_features=series_feature_names, show_result=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49855 entries, 0 to 49854
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Local_X(t-3)  49855 non-null  float64
 1   v_Vel(t-3)    49855 non-null  float64
 2   Local_Y(t-3)  49855 non-null  float64
 3   v_Acc(t-3)    49855 non-null  float64
 4   Lane_ID(t-3)  49855 non-null  float64
 5   Local_X(t-2)  49855 non-null  float64
 6   v_Vel(t-2)    49855 non-null  float64
 7   Local_Y(t-2)  49855 non-null  float64
 8   v_Acc(t-2)    49855 non-null  float64
 9   Lane_ID(t-2)  49855 non-null  float64
 10  Local_X(t-1)  49855 non-null  float64
 11  v_Vel(t-1)    49855 non-null  float64
 12  Local_Y(t-1)  49855 non-null  float64
 13  v_Acc(t-1)    49855 non-null  float64
 14  Lane_ID(t-1)  49855 non-null  float64
 15  Vehicle_ID    49855 non-null  int64  
 16  Total_Frames  49855 non-null  int64  
 17  v_Length      49855 non-null  float64
 18  v_Width       49855 non-nu

### Split the data set


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                                    test_size=0.3, random_state=42)
print(X_train.shape,X_test.shape, y_train.shape, y_test.shape)

(34898, 20) (14957, 20) (34898, 2) (14957, 2)


In [16]:
X_train.describe()

,Local_X(t-3),v_Vel(t-3),Local_Y(t-3),v_Acc(t-3),Lane_ID(t-3),Local_X(t-2),v_Vel(t-2),Local_Y(t-2),v_Acc(t-2),Lane_ID(t-2),Local_X(t-1),v_Vel(t-1),Local_Y(t-1),v_Acc(t-1),Lane_ID(t-1),Vehicle_ID,Total_Frames,v_Length,v_Width,v_Class
count,34898.000000,34898.000000,34898.000000,34898.000000,34898.000000,34898.000000,34898.000000,34898.000000,34898.000000,34898.000000,34898.000000,34898.000000,34898.000000,34898.000000,34898.000000,34898.000000,34898.000000,34898.000000,34898.000000,34898.000000
mean,29.825893,39.601055,979.097874,0.378789,2.990888,29.826814,39.639846,983.060561,0.387917,2.990372,29.827846,39.678644,987.027129,0.387981,2.989971,1500.487306,540.138489,15.056063,6.262224,2.019686
std,16.917318,13.716168,592.015808,5.531700,1.500046,16.911918,13.736282,592.690677,5.556588,1.499833,16.906664,13.756811,593.368784,5.512740,1.499735,775.945920,129.800916,5.651160,0.976939,0.138921
min,2.365371,0.000000,28.339231,-112.779778,1.000000,2.361841,0.000000,31.360769,-112.779778,1.000000,2.366730,0.000000,34.549814,-110.969666,1.000000,25.000000,283.000000,9.000000,4.900000,2.000000
25%,17.287998,32.172082,459.780124,-1.679239,2.000000,17.290910,32.158581,463.318751,-1.685207,2.000000,17.303439,32.214670,466.749357,-1.679907,2.000000,871.000000,452.000000,12.500000,5.500000,2.000000
50%,30.156991,40.066150,943.758431,0.041234,3.000000,30.159845,40.076013,947.272930,0.043439,3.000000,30.156510,40.082277,950.452304,0.042833,3.000000,1556.000000,522.000000,14.500000,6.000000,2.000000
75%,41.948858,49.430577,1458.548900,2.783238,4.000000,41.955640,49.513019,1463.448717,2.797432,4.000000,41.957030,49.541100,1467.941622,2.791441,4.000000,2171.000000,599.000000,16.500000,6.900000,2.000000
max,72.084203,83.660956,2128.450341,74.070325,8.000000,71.755825,83.660956,2134.345627,79.159472,8.000000,71.420084,83.660956,2140.250014,90.621110,8.000000,2761.000000,920.000000,61.500000,8.500000,3.000000


In [17]:
### Standardize the data
train_mean = X_train.mean()
train_std = X_train.std()

X_train = (X_train - train_mean) / train_std
X_test = (X_test - train_mean) / train_std

print(X_train.shape)
X_train.describe()

(34898, 20)


,Local_X(t-3),v_Vel(t-3),Local_Y(t-3),v_Acc(t-3),Lane_ID(t-3),Local_X(t-2),v_Vel(t-2),Local_Y(t-2),v_Acc(t-2),Lane_ID(t-2),Local_X(t-1),v_Vel(t-1),Local_Y(t-1),v_Acc(t-1),Lane_ID(t-1),Vehicle_ID,Total_Frames,v_Length,v_Width,v_Class
count,3.489800e+04,3.489800e+04,3.489800e+04,3.489800e+04,3.489800e+04,3.489800e+04,3.489800e+04,3.489800e+04,3.489800e+04,3.489800e+04,3.489800e+04,3.489800e+04,3.489800e+04,3.489800e+04,3.489800e+04,3.489800e+04,3.489800e+04,3.489800e+04,3.489800e+04,3.489800e+04
mean,-4.418241e-17,-9.752707e-17,-1.759152e-16,-2.443267e-18,-1.384518e-16,8.836482e-17,7.065113e-17,-1.828378e-16,3.054084e-19,-1.504645e-16,-1.708251e-16,-5.232663e-16,-3.094805e-17,1.165642e-17,1.738792e-16,-6.230331e-17,-2.505367e-16,-1.290859e-16,-9.986853e-16,1.684836e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.623220e+00,-2.887181e+00,-1.605968e+00,-2.045638e+01,-1.327217e+00,-1.624001e+00,-2.885777e+00,-1.605728e+00,-2.036640e+01,-1.327062e+00,-1.624278e+00,-2.884291e+00,-1.605203e+00,-2.020006e+01,-1.326882e+00,-1.901534e+00,-1.981022e+00,-1.071650e+00,-1.394380e+00,-1.417063e-01
25%,-7.411278e-01,-5.416216e-01,-8.772025e-01,-3.720426e-01,-6.605714e-01,-7.412467e-01,-5.446354e-01,-8.769192e-01,-3.730930e-01,-6.603215e-01,-7.407970e-01,-5.425657e-01,-8.768203e-01,-3.751108e-01,-6.600972e-01,-8.112515e-01,-6.790283e-01,-4.523077e-01,-7.802166e-01,-1.417063e-01
50%,1.957154e-02,3.390854e-02,-5.969341e-02,-6.102187e-02,6.074659e-03,1.969211e-02,3.175291e-02,-6.038163e-02,-6.199461e-02,6.419421e-03,1.943997e-02,2.934055e-02,-6.163928e-02,-6.260915e-02,6.687334e-03,7.154196e-02,-1.397408e-01,-9.839809e-02,-2.684140e-01,-1.417063e-01
75%,7.166009e-01,7.166377e-01,8.098619e-01,4.346674e-01,6.727207e-01,7.171763e-01,7.187661e-01,8.105209e-01,4.336320e-01,6.731603e-01,7.174203e-01,7.169144e-01,8.104816e-01,4.359827e-01,6.734719e-01,8.641230e-01,4.534753e-01,2.555115e-01,6.528306e-01,-1.417063e-01
max,2.497932e+00,3.212260e+00,1.941422e+00,1.332168e+01,3.339305e+00,2.479258e+00,3.204733e+00,1.942472e+00,1.417625e+01,3.340124e+00,2.460109e+00,3.197130e+00,1.943518e+00,1.636811e+01,3.340610e+00,1.624485e+00,2.926493e+00,8.218478e+00,2.290599e+00,7.056645e+00


### Reshape data sets


In [18]:
X_train = X_train.values
X_test = X_test.values
# reshape into [# samples, # timesteps,# features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],1))
X_test = X_test.reshape((X_test.shape[0], X_train.shape[1],1))

## Prediction model

In [19]:
%%time
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1],1)))
model.add(Dense(n_labels*n_future))
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

# For saving the best model during the whole training process.
#checkpointer = callbacks.ModelCheckpoint(filepath='BestModel.h5', monitor='val_loss', save_best_only=True)

#### Interrupt training if `val_loss` stops improving for over 10 epochs #######
stop_learn= tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_loss')


# fit model
Monitor = model.fit(X_train, y_train, epochs=50, 
                    callbacks=[stop_learn],
                    validation_data=(X_test, y_test), verbose=1)

Train on 34898 samples, validate on 14957 samples
Epoch 1/50
   32/34898 [..............................] - ETA: 2:12:00WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: 


InternalError:  Blas GEMM launch failed : a.shape=(32, 50), b.shape=(50, 200), m=32, n=200, k=50
	 [[{{node sequential/lstm/while/body/_1/MatMul_1}}]] [Op:__inference_distributed_function_1865]

Function call stack:
distributed_function


In [20]:
hist = pd.DataFrame(Monitor.history)
hist['epoch'] = Monitor.epoch
fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(10,4),dpi=150)
hist[['loss','val_loss']].plot(ax=axes[0])
hist[['mse','val_mse']].plot(ax=axes[1])
plt.show()
hist.tail()

NameError: name 'Monitor' is not defined

## Evaluation 


In [ ]:
yhat = model.predict(X_test, verbose=1)
rms += [sqrt(mean_squared_error(y_test, yhat))]
print(yhat[:5])
rms

In [ ]:
plt.figure(figsize=(n_labels*5,n_future*5))
k = 1
for i in range(n_future):
  for j in range(1,n_labels+1):
    plt.subplot(n_labels,n_future,k)
    plt.scatter(y_test.index,y_test.iloc[:,i], label = "true {} at t+{}".format(target_names[j-1],i),marker = 'X', )
    plt.scatter(y_test.index,yhat[:,i], label = "prediction {} at t+{}".format(target_names[j-1],i),marker = '.')
    plt.legend()
    k+=1
plt.show()

#END
